In [80]:
import pandas as pd 
dataset = pd.read_csv('Train.csv')
print(dataset.shape)

(40000, 2)


In [81]:
grouped = dataset.groupby('label')

# Select 1000 random reviews from each category
dataset = pd.concat([g.sample(1000, random_state=42) for _, g in grouped])

In [82]:
import re
import nltk
nltk.download('stopwords')

In [83]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 2000):
  review = re.sub('[^a-zA-Z]', ' ', dataset.iloc[i, dataset.columns.get_loc('text')])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [84]:
#bag of words  model 

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [86]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 70.81 %
Standard Deviation: 3.79 %


In [87]:
import lightgbm as lgb

In [88]:
classifier = lgb.LGBMClassifier(max_depth=20, n_estimators=25, min_child_weight= 0.0016, n_jobs=-1)

In [89]:
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 77.38 %
Standard Deviation: 2.74 %


In [90]:
#using TF-IDF for vectorization instead of bag of words
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [92]:
classifier = lgb.LGBMClassifier(max_depth=20, n_estimators=25, min_child_weight= 0.0016, n_jobs=-1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 77.31 %
Standard Deviation: 2.73 %
